In [9]:
import pandas as pd
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os

In [10]:
# carregar variáveis de ambiente
load_dotenv()

True

In [11]:
# chave da API
api_key = os.getenv("OPENAI_API_KEY")

# Instanciar o modelo de linguagem com a chave da API
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo")

## Dados

In [28]:
arquivo = r"C:\Users\felip\Desktop\teste_ia\manifestacoes-ouvidoria.xlsx"

df = pd.read_excel(arquivo)
df

,PROTOCOLO,DESCRICAO DA MANIFESTACAO,sentimento,palavras_inapropriadas,resumo,sumarizacao
0,1234,"""Estou extremamente insatisfeito com o atendim...",NaN,False,O autor está muito insatisfeito com o atendime...,O autor está muito insatisfeito com o atendime...
1,1235,"""Gostaria de parabenizar a equipe pelo excelen...",NaN,False,A equipe foi elogiada pelo excelente atendimen...,A equipe foi parabenizada pelo excelente atend...
2,1236,"""O aplicativo do banco está muito instável. Se...",-0.7,False,Usuário relata instabilidade no aplicativo do ...,"O aplicativo do banco está com problemas, trav..."
3,1237,"""Estou satisfeito com os serviços do banco, ma...",0.4,False,O cliente está satisfeito com os serviços do b...,O autor está satisfeito com os serviços do ban...
4,1238,"""O gerente da minha agência foi muito atencios...",NaN,False,O gerente da agência foi muito prestativo e re...,O gerente da agência foi atencioso e resolveu ...
5,1239,"""Acho que o atendimento telefônico poderia ser...",-0.7,False,O autor do texto acredita que o atendimento te...,O autor do texto sugere que o atendimento tele...
6,1240,"""Não tenho do que reclamar, sempre fui bem ate...",NaN,False,O cliente está satisfeito com o atendimento e ...,O autor do texto está satisfeito com o atendim...
7,1241,"""Fui cobrado indevidamente em uma transação e,...",-0.8,False,O autor foi cobrado indevidamente em uma trans...,A pessoa foi cobrada indevidamente em uma tran...
8,1242,"""O sistema de segurança do banco me parece exc...",0.8,False,O autor considera o sistema de segurança do ba...,O sistema de segurança do banco é excelente e ...
9,1243,"""Estou neutro quanto ao atendimento. Fui atend...",0.0,False,"O cliente foi atendido de forma neutra, sem ne...","O atendimento foi neutro, sem destacar nenhuma..."


## Protocolos sem análise de IA

In [30]:
# filtra os protocolos que ainda não passaram pela IA
df = df[
    (df['DESCRICAO DA MANIFESTACAO'].notna()) & 
    (df['DESCRICAO DA MANIFESTACAO'] != '') & 
    (df['resumo'].isna() | (df['resumo'] == ''))
]
df

,PROTOCOLO,DESCRICAO DA MANIFESTACAO,sentimento,palavras_inapropriadas,resumo,sumarizacao


## Prompts

### Análise de sentimento

In [13]:
sentimento_template = PromptTemplate(
    input_variables=["texto"],
    template="""
    Analise o sentimento do seguinte texto e forneça uma pontuação numérica entre -1 (muito negativo) e 1 (muito positivo).

    Texto:
    {texto}

    Pontuação:
    """
)

### Detecção de palavras inapropriadas

In [14]:
palavras_inapropriadas_template = PromptTemplate(
    input_variables=["texto"],
    template="""
    O texto a seguir contém palavras inapropriadas? Responda apenas 'Sim' ou 'Não'.

    Texto:
    {texto}

    Resposta:
    """
)

### Resumo da manifestação

In [15]:
resumo_template = PromptTemplate(
    input_variables=["texto"],
    template="""
    Resuma o seguinte texto em português.

    Texto:
    {texto}

    Resumo:
    """
)

## Chains para cada tarefa

In [17]:
sentimento_chain = LLMChain(llm=llm, prompt=sentimento_template)
palavras_inapropriadas_chain = LLMChain(llm=llm, prompt=palavras_inapropriadas_template)
resumo_chain = LLMChain(llm=llm, prompt=resumo_template)

C:\Users\felip\AppData\Local\Temp\ipykernel_23856\1838828182.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  sentimento_chain = LLMChain(llm=llm, prompt=sentimento_template)


## Funções

In [18]:
# Função para análise de sentimento
def analisar_sentimento(texto):
    resposta = sentimento_chain.invoke({"texto": texto})
    resposta_texto = resposta["text"].strip()  # extrair texto do dicionário
    try:
        score = float(resposta_texto)
    except ValueError:
        score = None
    return score

# Função para detecção de palavras inapropriadas
def detectar_palavras_inapropriadas(texto):
    resposta = palavras_inapropriadas_chain.invoke({"texto": texto})
    resposta_texto = resposta["text"].strip()
    return resposta_texto.lower() == 'sim'

# Função para sumarização
def resumir_texto(texto):
    resposta = resumo_chain.invoke({"texto": texto})
    resposta_texto = resposta["text"].strip()
    return resposta_texto

## Execução

In [19]:
# novas colunas no df
df['sentimento'] = None
df['palavras_inapropriadas'] = None
df['sumarizacao'] = None

# loop no df
for index, row in df.iterrows():
    texto = row['DESCRICAO DA MANIFESTACAO']
    
    # Analise de sentimento
    score = analisar_sentimento(texto)
    df.at[index, 'sentimento'] = score
    
    # Detectar palavras inapropriadas
    contem_inapropriadas = detectar_palavras_inapropriadas(texto)
    df.at[index, 'palavras_inapropriadas'] = contem_inapropriadas
    
    # Sumarização
    resumo_sentimento = resumir_texto(texto)
    df.at[index, 'sumarizacao'] = resumo_sentimento

In [20]:
df

,PROTOCOLO,DESCRICAO DA MANIFESTACAO,sentimento,palavras_inapropriadas,resumo,sumarizacao
0,1234,"""Estou extremamente insatisfeito com o atendim...",None,False,O autor está muito insatisfeito com o atendime...,O autor está muito insatisfeito com o atendime...
1,1235,"""Gostaria de parabenizar a equipe pelo excelen...",None,False,A equipe foi elogiada pelo excelente atendimen...,A equipe foi parabenizada pelo excelente atend...
2,1236,"""O aplicativo do banco está muito instável. Se...",-0.7,False,Usuário relata instabilidade no aplicativo do ...,"O aplicativo do banco está com problemas, trav..."
3,1237,"""Estou satisfeito com os serviços do banco, ma...",0.4,False,O cliente está satisfeito com os serviços do b...,O autor está satisfeito com os serviços do ban...
4,1238,"""O gerente da minha agência foi muito atencios...",None,False,O gerente da agência foi muito prestativo e re...,O gerente da agência foi atencioso e resolveu ...
5,1239,"""Acho que o atendimento telefônico poderia ser...",-0.7,False,O autor do texto acredita que o atendimento te...,O autor do texto sugere que o atendimento tele...
6,1240,"""Não tenho do que reclamar, sempre fui bem ate...",None,False,O cliente está satisfeito com o atendimento e ...,O autor do texto está satisfeito com o atendim...
7,1241,"""Fui cobrado indevidamente em uma transação e,...",-0.8,False,O autor foi cobrado indevidamente em uma trans...,A pessoa foi cobrada indevidamente em uma tran...
8,1242,"""O sistema de segurança do banco me parece exc...",0.8,False,O autor considera o sistema de segurança do ba...,O sistema de segurança do banco é excelente e ...
9,1243,"""Estou neutro quanto ao atendimento. Fui atend...",0.0,False,"O cliente foi atendido de forma neutra, sem ne...","O atendimento foi neutro, sem destacar nenhuma..."


In [21]:
# Exportar
df.to_excel(r"C:\Users\felip\Desktop\teste_ia\manifestacoes-ouvidoria.xlsx", index=False)